# Amazon RDS (Relational Database Service)

<img src="../_assets/aws_service_icons/rds.svg" width="80" alt="Amazon RDS">

## Goals
- Understand what **Amazon RDS** is (and what it is not).
- Know when RDS is a good practical choice.
- See a minimal **AWS SDK** pseudo-code workflow (no execution).


## Prerequisites
- Basic SQL + relational concepts (tables, indexes, transactions).
- Basic AWS networking concepts help (VPC, subnets, security groups).

> This notebook includes **pseudo-code only**. It does not run any AWS SDK calls.


## What RDS is
**Amazon RDS** is AWS’s managed service for running **relational databases**.

Instead of you provisioning a VM, installing a database, patching it, backing it up, and setting up replication, RDS provides managed primitives like:
- **DB instances** (compute + storage for a database engine)
- **Automated backups** + **point-in-time restore**
- **Snapshots** (manual backups)
- **High availability** options (e.g., **Multi-AZ** deployments)
- **Read replicas** (engine-dependent)
- Monitoring/metrics (CloudWatch), maintenance windows, parameter groups

RDS supports common engines such as **PostgreSQL**, **MySQL**, **MariaDB**, **Oracle**, **SQL Server**, and also **Amazon Aurora** (a managed, RDS-compatible engine).

### What it is not
- Not a data lake (that’s typically S3-based).
- Not a distributed OLAP warehouse by default (for that, consider services like Redshift).
- Not “serverless SQL” by default (though some engine options provide serverless/autoscaling modes).


## What RDS is practically used for
RDS is commonly used when you need a **durable, transactional (OLTP) database** without running the operational burden yourself.

Typical use-cases:
- **Application backends**: user accounts, orders, payments, inventory, session/state.
- **Metadata stores**: tracking datasets, experiments, jobs, model versions, feature definitions.
- **Feature serving (lightweight)**: storing curated features for online inference when latency is important and the dataset fits relational access patterns.
- **Internal tools**: dashboards, admin panels, workflow orchestration state.

Operational reasons to choose RDS:
- You want managed backups, patching, monitoring, and HA options.
- You want predictable relational semantics (constraints, joins, ACID transactions).
- You want standard SQL drivers/tools (psql, mysql client, ORM tooling).


## Using RDS with the AWS SDK (pseudo-code)
Below is a minimal, **non-executable** sketch of how you might create an RDS instance, wait for it to be available, and fetch its endpoint using the AWS SDK.

Notes:
- In real projects, do **not** hardcode passwords; use **AWS Secrets Manager** or IAM auth where supported.
- Use a **private subnet group** + security groups (avoid public databases unless required).
- Creating RDS resources can incur cost; always plan cleanup.

```python
# PSEUDO-CODE (do not run)

import boto3

region = "us-east-1"
rds = boto3.client("rds", region_name=region)

# 1) Create (or reference) a DB subnet group + security group beforehand.
db_subnet_group_name = "my-private-subnets"
vpc_security_group_ids = ["sg-xxxxxxxx"]

# 2) Create the DB instance.
db_id = "demo-postgres"
rds.create_db_instance(
    DBInstanceIdentifier=db_id,
    Engine="postgres",
    DBInstanceClass="db.t3.micro",
    AllocatedStorage=20,
    MasterUsername="admin",
    MasterUserPassword="<stored-in-secrets-manager>",
    DBSubnetGroupName=db_subnet_group_name,
    VpcSecurityGroupIds=vpc_security_group_ids,
    PubliclyAccessible=False,
    StorageEncrypted=True,
    MultiAZ=False,
)

# 3) Wait until the instance is ready.
waiter = rds.get_waiter("db_instance_available")
waiter.wait(DBInstanceIdentifier=db_id)

# 4) Fetch the endpoint to connect from your app (inside the VPC).
resp = rds.describe_db_instances(DBInstanceIdentifier=db_id)
db = resp["DBInstances"][0]
endpoint = db["Endpoint"]["Address"]
port = db["Endpoint"]["Port"]

print(f"Connect to: {endpoint}:{port}")

# 5) (Optional) Take a snapshot.
rds.create_db_snapshot(
    DBSnapshotIdentifier=f"{db_id}-snapshot-001",
    DBInstanceIdentifier=db_id,
)

# 6) (Optional) Clean up to avoid ongoing charges.
# rds.delete_db_instance(DBInstanceIdentifier=db_id, SkipFinalSnapshot=False)
```
